In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("proj3.ipynb")

# Project 3: Data Transformation

## Due Date: Wednesday, April 3rd, 5:00 PM

## Assignment Details

In this project, we'll be working with one month of data from sensors in buildings at UC Berkeley. This is a very typical real-world dataset—i.e. it's kind of a mess. The full dataset contains a giant `data` table of many billions of sensor readings over the course of a decade; we will look at a single month of that data. It also contains a variety of other tables that contextualize the readings.

The **schema for the database** is shown below.
* Each line represents a relationship between the two fields.
* The side of the line diverging to three lines / arrows represents the **"many"** side of the relationship, while the side of the line converging to one arrow represents the **"one"** side of the relationship.
* This file is available as `data/schema.png`.

Occasionally people believe that when the data has a well-structured schema, then it's simple to proceed! *We'll put this assumption to the test.*

Citation: Luo, N., Wang, Z., Blum, D. et al. A three-year dataset supporting research on building energy management and occupancy analytics. _Sci Data_ 9, 156 (2022). https://doi.org/10.1038/s41597-022-01257-x 

<img src="data/schema.png">

If you make any queries to explore the data, do not forget to add a LIMIT clause; LIMIT 10 is a good default to always add to the end of your queries. Otherwise, your connection may close as a result of trying to load excessively many rows. To explore the data within the JupyterNotebook, you may add a cell like so:
```
%%sql
SELECT ... 
LIMIT 10;
```
In addition, for the entirety of the project, you may make as many CTEs as you'd like.


## Logistics & Scoring Breakdown

This is an **individual project**. However, you’re welcome to collaborate with any other student in the class as long as it’s within the academic honesty guidelines. Free-response questions (marked 'm' in the table below) are manually graded.

Question | Points
--- | ---
1a	| m: 1
1b  | 1
1c	| 1
1d	| m: 2
2a	| 3
2b	| 1
3a	| m: 1
3b	| 1
3c	| 1
3d  | m: 1
3e  | 2
4a	| 2
4b	| 2
4c	| 3
5a  | 1
5b  | 1
5c  | 3
**Total** | 27

**Grand Total:** 27 points (autograded: 22, manual: 5)

In [2]:
# Run this cell to set up imports
import numpy as np
import pandas as pd

In [3]:
%reload_ext sql
%sql postgresql://jovyan@127.0.0.1:5432/postgres

## Loading Up the Database
To load the database, run the following cell.

In [4]:
# This cell sets up the database

# first clean existing connections
import subprocess
call = subprocess.run(["psql", "-h", "localhost", \
                       "-tAc", "SELECT 1 FROM pg_database WHERE datname='ucb_buildings'", "template1"], \
                      stdout=subprocess.PIPE, text=True)
if call.stdout == "1\n":
    !psql postgresql://localhost:5432/ucb_buildings -c 'SELECT pg_terminate_backend(pg_stat_activity.pid) FROM pg_stat_activity WHERE datname = current_database()  AND pid <> pg_backend_pid();'

# then delete and recreate database
!psql -h localhost -c 'DROP DATABASE IF EXISTS ucb_buildings'
!psql -h localhost -c 'CREATE DATABASE ucb_buildings'
!gunzip -c data/proj3.sql.gz | psql -h localhost -d ucb_buildings -f -

 pg_terminate_backend 
----------------------
(0 rows)

DROP DATABASE
CREATE DATABASE
SET
SET
SET
SET
SET
 set_config 
------------
 
(1 row)

SET
SET
SET
SET
CREATE EXTENSION
COMMENT
CREATE EXTENSION
COMMENT
CREATE FUNCTION
ALTER FUNCTION
CREATE AGGREGATE
ALTER AGGREGATE
SET
SET
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE MATERIALIZED VIEW
ALTER TABLE
CREATE TABLE
ALTER TABLE
COPY 92
COPY 102386
COPY 100000
COPY 381
COPY 9509
COPY 26183
COPY 5276
COPY 12
ALTER TABLE
ALTER TABLE
ALTER TABLE
REFRESH MATERIALIZED VIEW


Now, run the following cell to connect to the `ucb_buildings` database. There should be no errors after running the following cell.

In [5]:
%sql postgresql://jovyan@127.0.0.1:5432/ucb_buildings

Connecting and switching to connection postgresql://jovyan@127.0.0.1:5432/ucb_buildings

## Connect to the grader

Run the following cell for grading purposes.

In [6]:
# Just run the following cell, no further action is needed.
from data101_utils import GradingUtil
grading_util = GradingUtil("proj3")
grading_util.prepare_autograder()

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


<!-- BEGIN QUESTION -->

## Question 1: Unboxing the Data

### Question 1a

As mentioned above, we are working with just one month of data. In the full database (which we don't have access to), tables like the `data` table have billions of rows. What do you notice about the design of the database schema above that helps support the large amount of data and minimize redundancy? Keep your response to at most three sentences.

**Hint:** There is no need to examine any data here. What is a technique learned in lecture 16? Define that technique.

The design of the database schema is of a normalized design i.e. decomposing relations into multiple relations in order to remove redundancy, minimize update and delete anomolies. These decomposed relations are connected by foreign keys which allow us to use natural joins to recover the original relations. For example the data and metadata relations seperate the data value (data) from the descriptive information (metadata) which allow for smaller subsets of the combined relation which can be reconfigered by joining the two relations by the foreign key id.

<!-- END QUESTION -->

<br><br>

---
### Question 1b

The diagram claims that `buildings_site_mapping` has a **many-to-many relationship** with `real_estate_metadata`. Let's validate that. 

Below is an example of the aggregate function `json_agg` being used with a table; you will need to do this in the next two parts. Note the necessary use of a `GROUP BY` to use `json_agg`, which aggregates values from the table and returns a single JSON array. [(Documentation)](https://www.postgresql.org/docs/9.5/functions-aggregate.html)

In [7]:
%%sql
SELECT b.site, json_agg(b) from buildings_site_mapping b GROUP BY b.site LIMIT 5;

Running query in 'postgresql://jovyan@127.0.0.1:5432/ucb_buildings'

5 rows affected.

site,json_agg
Sutardja Dai Hall,"[{'site': 'Sutardja Dai Hall', 'building': 'SUTARDJA DAI'}]"
Le Conte Hall,"[{'site': 'Le Conte Hall', 'building': 'LE CONTE'}]"
Hildebrand Hall,"[{'site': 'Hildebrand Hall', 'building': 'HILDEBRA ND'}]"
Stephens Hall,"[{'site': 'Stephens Hall', 'building': 'STEPHENS'}]"
2000 Carleton Street,"[{'site': '2000 Carleton Street', 'building': 'CARLETO2 000'}]"


Find the **distinct** values of `buildings_site_mapping.building` that match multiple tuples in `real_estate_metadata.building_name`, and for each such value of `buildings_site_mapping.building` return the matches as JSON via `json_agg(real_estate_metadata)`. Your output should contain the building and the `json_agg` in that order. **Order your final result by building ascending**.

**Hint:** You should use a CTE to find the distinct buildings of `buildings_site_mapping` before applying necessary table joins.

In [8]:
%%sql --save query_1b result_1b <<
WITH distinct_building AS (SELECT DISTINCT(building) FROM buildings_site_mapping WHERE building IN (SELECT building_name FROM real_estate_metadata GROUP BY building_name HAVING COUNT(*) > 1))
SELECT d.building, json_agg(r) FROM distinct_building d JOIN real_estate_metadata r ON d.building = r.building_name GROUP BY d.building ORDER BY d.building;

Running query in 'postgresql://jovyan@127.0.0.1:5432/ucb_buildings'

5 rows affected.

In [9]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_1b = %sqlcmd snippets query_1b
grading_util.save_results("result_1b", query_1b, result_1b)
result_1b

building,json_agg
ALUMNI HOUSE,"[{'location': 'IRVINE', 'building': '9207', 'building_name': 'ALUMNI HOUSE', 'address': 'CORE CAMPUS', 'city_name': 'IRVINE', 'county': 'ORANGE', 'category': 'GENERAL', 'osfg': 4027, 'bldg_asf': 2549, 'levels': 1, 'year': '1984', 'owner': 'UC', 'mp_code': 'P', 'book_value': 41981}, {'location': 'FRANCISC SOAN', 'building': '2032', 'building_name': 'ALUMNI HOUSE', 'address': 'PARNASS US AVE', 'city_name': 'FRANCISC OSAN', 'county': 'FRANCISC OSAN', 'category': 'HEALTH SCIENCE', 'osfg': 7217, 'bldg_asf': 5079, 'levels': 3, 'year': '1915', 'owner': 'UP', 'mp_code': 'P', 'book_value': 135923}, {'location': 'BERKELEY', 'building': '1215', 'building_name': 'ALUMNI HOUSE', 'address': 'CORE C22A2M0PUS', 'city_name': 'BERKELEY', 'county': 'ALAMEDA', 'category': 'GENERAL', 'osfg': 15590, 'bldg_asf': 8719, 'levels': 2, 'year': '1954', 'owner': 'UC', 'mp_code': 'P', 'book_value': 106819}]"
CAMPBELL,"[{'location': 'BERKELEY', 'building': '1027', 'building_name': 'CAMPBELL', 'address': 'CORE C28A5M0PUS', 'city_name': 'BERKELEY', 'county': 'ALAMEDA', 'category': 'GENERAL', 'osfg': 83373, 'bldg_asf': 50745, 'levels': 7, 'year': '2014', 'owner': 'UC', 'mp_code': 'P', 'book_value': 75256344.0}, {'location': 'LOS ANGELES', 'building': '4294', 'building_name': 'CAMPBELL', 'address': 'PORTOLA P11L8A1Z1A', 'city_name': 'LOS ANGELES', 'county': 'LOS ANGELES', 'category': 'GENERAL', 'osfg': 55401, 'bldg_asf': 32619, 'levels': 5, 'year': '1954', 'owner': 'UC', 'mp_code': 'P', 'book_value': 6247192.0}]"
HERTZ,"[{'location': 'BERKELEY', 'building': '1423', 'building_name': 'HERTZ', 'address': 'CORE CAMPUS', 'city_name': 'BERKELEY', 'county': 'ALAMEDA', 'category': 'GENERAL', 'osfg': 31362, 'bldg_asf': 13518, 'levels': 4, 'year': '1958', 'owner': 'UC', 'mp_code': 'P', 'book_value': 3491181.0}, {'location': 'DAVIS', 'building': '9454', 'building_name': 'HERTZ', 'address': '7000 EAST 2A4V0E0 LAKE', 'city_name': 'LIVERMOR E', 'county': 'ALAMEDA', 'category': 'GENERAL', 'osfg': 10904, 'bldg_asf': 8183, 'levels': 1, 'year': '1976', 'owner': 'UC', 'mp_code': 'P', 'book_value': 0}]"
SOUTH,"[{'location': 'BERKELEY', 'building': '1484', 'building_name': 'SOUTH', 'address': 'CORE CAMPUS', 'city_name': 'BERKELEY', 'county': 'ALAMEDA', 'category': 'GENERAL', 'osfg': 30401, 'bldg_asf': 15473, 'levels': 4, 'year': '1873', 'owner': 'UC', 'mp_code': 'P', 'book_value': 1426174.0}, {'location': 'DAVIS', 'building': '3275', 'building_name': 'SOUTH', 'address': 'CORE CAMPUS', 'city_name': 'DAVIS (YOLO)', 'county': 'YOLO', 'category': 'GENERAL', 'osfg': 26594, 'bldg_asf': 15323, 'levels': 4, 'year': '1912', 'owner': 'UC', 'mp_code': 'P', 'book_value': 3585399.0}]"
SPROUL,"[{'location': 'BERKELEY', 'building': '1210', 'building_name': 'SPROUL', 'address': 'CORE 2C6A0M0PUS', 'city_name': 'BERKELEY', 'county': 'ALAMEDA', 'category': 'GENERAL', 'osfg': 111197, 'bldg_asf': 66680, 'levels': 6, 'year': '1941', 'owner': 'UC', 'mp_code': 'P', 'book_value': 4278819.0}, {'location': 'RIVERSIDE', 'building': 'P5523', 'building_name': 'SPROUL', 'address': 'CORE CAMPUS', 'city_name': 'RIVERSIDE', 'county': 'RIVERSIDE', 'category': 'GENERAL', 'osfg': 80988, 'bldg_asf': 45532, 'levels': 5, 'year': '1965', 'owner': 'UC', 'mp_code': 'P', 'book_value': 1296147.0}, {'location': 'DAVIS', 'building': '3815', 'building_name': 'SPROUL', 'address': 'CORE CAMPUS', 'city_name': 'DAVIS (YOLO)', 'county': 'YOLO', 'category': 'GENERAL', 'osfg': 53874, 'bldg_asf': 27913, 'levels': 10, 'year': '1963', 'owner': 'UC', 'mp_code': 'P', 'book_value': 1620880.0}]"


In [10]:
grader.check("q1b")

q1b results: All test cases passed!

<br><br>

---
### Question 1c

Now find examples of many matches in the opposite direction. For each distinct `real_estate_metadata.building_name` value, find the ones that have multiple matches in `buildings_site_mapping.building`, and for each return a `json_agg` of the multiple values for `buildings_site_mapping`. Your output should contain the building name and the `json_agg` in that order. **Order your final result by building name ascending.**

**Hint:** You should use a CTE to find the distinct building names of `real_estate_metadata` before applying necessary table joins.

In [11]:
%%sql --save query_1c result_1c <<
WITH distinct_building AS (SELECT DISTINCT(building_name) FROM real_estate_metadata WHERE building_name IN (SELECT building FROM buildings_site_mapping GROUP BY building HAVING COUNT(*) > 1))
SELECT d.building_name, json_agg(b) FROM distinct_building d JOIN buildings_site_mapping b ON d.building_name = b.building GROUP BY d.building_name ORDER BY d.building_name;

Running query in 'postgresql://jovyan@127.0.0.1:5432/ucb_buildings'

4 rows affected.

In [12]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_1c = %sqlcmd snippets query_1c
grading_util.save_results("result_1c", query_1c, result_1c)
result_1c

building_name,json_agg
EDWARDS FLD,"[{'site': 'Edwards Stadium East', 'building': 'EDWARDS FLD'}, {'site': 'Edwards Stadium West', 'building': 'EDWARDS FLD'}]"
FAC CLUB,"[{'site': ""Men's Faculty Club"", 'building': 'FAC CLUB'}, {'site': 'Womens Faculty Club', 'building': 'FAC CLUB'}]"
HAAS STU BLD,"[{'site': 'Haas School Student Services (Water)', 'building': 'HAAS STU BLD'}, {'site': 'Haas School of Business', 'building': 'HAAS STU BLD'}]"
SIMON,"[{'site': 'Simpson Center', 'building': 'SIMON'}, {'site': 'Boalt and Simon Hall', 'building': 'SIMON'}]"


In [13]:
grader.check("q1c")

q1c results: All test cases passed!

<!-- BEGIN QUESTION -->

<br><br>

---
### Question 1d

Do you see any issues with the schema given? In particular, please address the two questions below:
- Can you uniquely determine the building given the sensor data? Why? (**Hint:** given a row in the `data` table, can you determine a **uniquely** associated row in `real_estate_metadata` table? Your answer should draw insights from 1b.)
- Could `buildings_site_mapping.building` be a valid foreign key pointing to `real_estate_metadata.building_name`? (**Hint:** think about the definition / constraints of a foreign key.)

Please keep your response to **at most three sentences.**


Determining a uniquely associated row in the real_estate_metadata table proves challenging due to multiple buildings sharing the same building_name despite being distinct structures in different universities. This complexity presents difficulty in pinpointing the specific building from sensor data, as observed in 1b where certain buildings in real_estate_metadata possess multiple tuples yet represent different structures with identical names. Therefore, establishing a foreign key relationship between buildings_site_mapping.building and real_estate_metadata.building_name is not feasible, as a single buildings_site_mapping.building may reference multiple building_names, each corresponding to distinct building sensors.

<!-- END QUESTION -->

<br/><br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Question 2: Looking for Outliers in the Readings
Physical sensors, such as the ones responsible for generating this data, are notorious for occasionally producing crazy outliers. In this section, we will undergo some data cleaning to address these outliers.

The readings from all different types of sensors are mixed together within this `data` table. This assortment of mixed readings will require some extra work to identify the outliers. Let's get started.

### Question 2a: Outlier Detection

Let's start with finding the outlying values *for each sensor id*. We'll define an outlier as an observation that is more than **3 Hampel X84 intervals** away from the median. See [lecture 14](https://docs.google.com/presentation/d/1vG53ZkZfwWm2Bz2QFCHcyB5ySuUod5ESYgRKz0qXqjk/edit?usp=sharing) for more.

Create a view `labeled_data` that contains all of the columns in `data` and adds three additional columns on the far right ***for each sensor id***:
  - `median` containing the median using `percentile_disc`
  - `mad` containing the Median Absolute Deviation (MAD),
  - `outlier` that contains `true` for the outlier readings and `false` for the rest. **Also,** for data points where the `mad` is 0, set this to `false`.

**Hint:** You should use CTEs to first compute the median and MAD values. Feel free to create as many CTEs as you need to complete the objective. Our staff solution used 3: one to compute the median of *each* sensor id, one to compute the absolute differences for each data point relative to its sensor id median, and another to finally compute the median of these absolute differences.

In [14]:
%%sql --save query_2a result_2a <<
CREATE OR REPLACE VIEW labeled_data AS
WITH median_id AS (
    SELECT id, percentile_disc(0.5) WITHIN GROUP (ORDER BY value) AS median FROM data GROUP BY id),
abs_dif_id AS (
    SELECT d.id, ABS(m.median - d.value) AS abs_dif FROM median_id m JOIN data d ON m.id = d.id),
mad_id AS (
    SELECT id, percentile_disc(0.5) WITHIN GROUP (ORDER BY abs_dif) AS mad FROM abs_dif_id GROUP BY id)
SELECT d.*, m.median, mad.mad,
    CASE 
        WHEN mad.mad = 0 THEN FALSE
        WHEN ABS(d.value - m.median) > (3 * mad.mad * 1.4826) THEN TRUE
        ELSE FALSE
    END AS outlier
    FROM data d JOIN median_id m ON d.id = m.id JOIN mad_id mad ON d.id = mad.id

;
SELECT * FROM labeled_data WHERE outlier ORDER BY id, time LIMIT 100;

Running query in 'postgresql://jovyan@127.0.0.1:5432/ucb_buildings'

100 rows affected.

In [15]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_2a = %sqlcmd snippets query_2a
grading_util.save_results("result_2a", query_2a, result_2a)
result_2a

time,id,value,median,mad,outlier
2018-06-09 08:45:00+00:00,a47e9bd4-ab61-56d5-9e0e-2dce29a46dd3,98.7,37.2,12.5,True
2018-06-07 00:00:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,812.6,727.5,16.200000000000045,True
2018-06-07 18:00:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,807.3,727.5,16.200000000000045,True
2018-06-07 18:15:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,800.0,727.5,16.200000000000045,True
2018-06-07 18:30:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,819.5,727.5,16.200000000000045,True
2018-06-07 18:45:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,808.7,727.5,16.200000000000045,True
2018-06-07 19:15:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,811.1,727.5,16.200000000000045,True
2018-06-07 19:30:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,804.2,727.5,16.200000000000045,True
2018-06-07 19:45:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,810.9,727.5,16.200000000000045,True
2018-06-07 20:00:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,801.7,727.5,16.200000000000045,True


In [16]:
grader.check("q2a")

q2a results: All test cases passed!

<br><br>

---

### Question 2b: Outlier Handling (Winsorization)

In this step, define a view `cleaned_data` containing all the columns of `labeled_data` and one additional column on the far right called `clean_value`. This column will contain a copy of `labeled_data.value` if that value is not an outlier. For outliers, it should contain the value Winsorized to the nearest outlier boundary value (3 Hampel X84 intervals from the median). If the MAD is 0, then the cleaned value should be unchanged (i.e., the same as the original value).

To reiterate, we consider a value to be an outlier if it is more than (strictly greater than or strictly less than) 3 Hampel X84 intervals from the median. 

In [17]:
%%sql --save query_2b result_2b <<
CREATE OR REPLACE VIEW cleaned_data AS
SELECT *, CASE
                           WHEN outlier = TRUE AND mad > 0 THEN 
                               CASE
                                   WHEN value > median + (3 * mad * 1.4826) THEN median + (3 * mad * 1.4826)
                                   WHEN value < median - (3 * mad * 1.4826) THEN median - (3 * mad * 1.4826)
                                   ELSE value
                               END
                           ELSE value
                       END AS clean_value
                   FROM labeled_data
;
SELECT * FROM cleaned_data WHERE outlier ORDER BY id, time LIMIT 100;

Running query in 'postgresql://jovyan@127.0.0.1:5432/ucb_buildings'

100 rows affected.

In [18]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_2b = %sqlcmd snippets query_2b
grading_util.save_results("result_2b", query_2b, result_2b)
result_2b

time,id,value,median,mad,outlier,clean_value
2018-06-09 08:45:00+00:00,a47e9bd4-ab61-56d5-9e0e-2dce29a46dd3,98.7,37.2,12.5,True,92.7975
2018-06-07 00:00:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,812.6,727.5,16.200000000000045,True,799.5543600000002
2018-06-07 18:00:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,807.3,727.5,16.200000000000045,True,799.5543600000002
2018-06-07 18:15:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,800.0,727.5,16.200000000000045,True,799.5543600000002
2018-06-07 18:30:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,819.5,727.5,16.200000000000045,True,799.5543600000002
2018-06-07 18:45:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,808.7,727.5,16.200000000000045,True,799.5543600000002
2018-06-07 19:15:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,811.1,727.5,16.200000000000045,True,799.5543600000002
2018-06-07 19:30:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,804.2,727.5,16.200000000000045,True,799.5543600000002
2018-06-07 19:45:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,810.9,727.5,16.200000000000045,True,799.5543600000002
2018-06-07 20:00:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,801.7,727.5,16.200000000000045,True,799.5543600000002


In [19]:
grader.check("q2b")

q2b results: All test cases passed!

<!-- BEGIN QUESTION -->

<br/><br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Question 3: Entity Resolution

### Question 3a
There is a lot of mess in this dataset related to entity names. As a start, have a look at all of the distinct values in the `units` field of the `metadata` table. What do you notice about these values? Are there any duplicates? **Limit your response to one sentence.**


In [20]:
%%sql
SELECT DISTINCT(units) FROM metadata;

Running query in 'postgresql://jovyan@127.0.0.1:5432/ucb_buildings'

34 rows affected.

units
Gal
subinterval
kVA
Hz
Gallons
kVAh
Bottom
Lbs
kVARh
uS


The units vary in terms of type of measure such as Lbs for weight and Hz for frequency and there are duplicate units such as Gal and Gallons.

<!-- END QUESTION -->

<br><br>

---

### Question 3b

Sometimes entity resolution is as simple as a text transformation. For example, how many unique `units` values are there, and how many would there be if we ignored case (upper vs. lower case)? Your output should be a table with one row and two columns; the first column should contain the number of unique `units` values, and the second column should contain the number of unique `units` values if we ignored case. The two columns can have arbitrary names—we will not be checking column names.


In [21]:
%%sql --save query_3b result_3b <<
SELECT COUNT(DISTINCT(units)), COUNT(DISTINCT(LOWER(units))) FROM metadata;

Running query in 'postgresql://jovyan@127.0.0.1:5432/ucb_buildings'

1 rows affected.

In [22]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_3b = %sqlcmd snippets query_3b
grading_util.save_results("result_3b", query_3b, result_3b)
result_3b

count,count_1
34,29


In [23]:
grader.check("q3b")

q3b results: All test cases passed!

<br><br>

---

### Question 3c

Arguably, we shouldn't care about these alternative unit labels, *as long as each sensor **class** uses a single value of `units` for all its sensor ids*. After all, maybe the capitalization means something to somebody!

Write a SQL query that returns a **single row with one column** of value `true` if the condition (in italics above) holds, or a single row with one column of value `false` otherwise. This column can have an arbitrary name—we will not be checking its name. Please do not hard code this query—we reserve the right to penalize your score if you do so.

**Hint:** You may find `ALL` or some other logical check such as `SUM(bool)  == len` to be useful here.



In [24]:
%%sql --save query_3c result_3c <<
WITH units AS (SELECT CASE
        WHEN LOWER(m1.units) = ALL (SELECT LOWER(m2.units) FROM metadata m2 WHERE m1.class = m2.class) THEN TRUE
        ELSE FALSE
        END AS units_test
FROM metadata m1)
SELECT CASE
        WHEN FALSE = ANY (SELECT u.units_test FROM units u) THEN FALSE
        ELSE TRUE
        END;

Running query in 'postgresql://jovyan@127.0.0.1:5432/ucb_buildings'

1 rows affected.

In [25]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_3c = %sqlcmd snippets query_3c
grading_util.save_results("result_3c", query_3c, result_3c)
result_3c

case
True


In [26]:
grader.check("q3c")

q3c results: All test cases passed!

<!-- BEGIN QUESTION -->

<br><br>

---

### Question 3d

Moving on, have a look at the `real_estate_metadata` table—starting with the distinct values in the `location` field! What do you notice about these values? Keep your response to at most two sentences.


In [27]:
%%sql 
SELECT DISTINCT(location) FROM real_estate_metadata;

Running query in 'postgresql://jovyan@127.0.0.1:5432/ucb_buildings'

15 rows affected.

location
LOS ANGELES
FRANCISC O
SYSTEMWI DE
SAN DSAIENG O
FRANCISC SOAN
SANTA CRUZ
AG FIELD STAT
IRVINE
RIVERSIDE
BERKELEY


There are spelling and spacing errors such as San Dsaieng o which I believe corresponds to San Diego. Furthermore, there are duplicate distinct locations due to spelling errors such as Francisc o and Francisc Soan which I believe both correspond to San Francisco which can lead to errors down the line if both San Francisco location values are not corrected.

<!-- END QUESTION -->

<br><br>

---
### Question 3e

It turns out this `real_estate_metadata` table is the result of an [OCR scan](https://en.wikipedia.org/wiki/Optical_character_recognition). We will just focus on cleaning up the `location` column for the time being, and leave you to imagine the effort required to do a full cleanup of all columns.

We have preloaded Postgres' extension packages for "fuzzy" string matching (`fuzzystrmatch`) and trigrams (`pg_trgm`) for you so that you can take advantage of some convenient utility functions. You can use any of the string functions in those packages if you'd like ([as documented here for fuzzystrmatch](https://www.postgresql.org/docs/current/fuzzystrmatch.html) or [here for pg_trgm](https://www.postgresql.org/docs/current/pgtrgm.html)).

We also created a lookup table of standardized names, `uc_locations`.

Now, using any of the string functions you like (or none at all!), write a SQL query that returns the columns `(building_name, address, location, clean_location)` where `clean_location` contains the best match from `uc_locations.loc_name`. While we recommend taking the time to explore different string functions such as `levenshtein`, `word_similarity`, and `metaphone`, we have found `word_similarity` to be the most powerful in this scenario. You may find that you can't clean up everything with the string functions, so your view may have to include some specific logic for cases in the data that have to be handled "manually". You can choose to do this question in whatever manner you wish as long as your query does not use `CREATE TABLE`, `INSERT INTO`, or `UPDATE`.

In [28]:
%%sql --save query_3e result_3e <<
SELECT r.building_name, r.address, r.location, l.loc_name AS clean_location FROM real_estate_metadata r,
LATERAL (
    SELECT loc_name FROM uc_locations ORDER BY word_similarity(r.location, loc_name) DESC LIMIT 1 ) l;

Running query in 'postgresql://jovyan@127.0.0.1:5432/ucb_buildings'

5276 rows affected.

In [29]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_3e = %sqlcmd snippets query_3e
grading_util.save_results("result_3e", query_3e, result_3e)
result_3e

building_name,address,location,clean_location
CLEARY,2424 CHANNING,BERKELEY,BERKELEY
WALNUT19 25,1925 WALNUT,BERKELEY,BERKELEY
ERCHLDE DCNTR,2339 HASTE,BERKELEY,BERKELEY
MRI MODLR II,CORE 2C1A5M4P-6U0S,BERKELEY,BERKELEY
UNIV 2154- 60,UNIVERSIT 2Y55,BERKELEY,BERKELEY
HAVENS,PANORAMI C25 W5 AY,BERKELEY,BERKELEY
HAVENS GR,PANORAMI 2C9 W99A Y,BERKELEY,BERKELEY
BOAT HOUSE,GLASCOC K STREET,BERKELEY,BERKELEY
BOAT EBRIGHT,303 DERBY AVENUE,BERKELEY,BERKELEY
BOAT SHED,303 DERBY AVENUE,BERKELEY,BERKELEY


In [30]:
grader.check("q3e")

q3e results: All test cases passed!

<br/><br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Question 4: Interpolating Missing Data
Real-world data, real-world problems. Our sensors should be reporting every 15 minutes, but you can be sure that we're missing some data. Here we will fix it. It's a bit more involved than what we looked at in class!

### Question 4a: Finding missing readings
In the `data` table, the `id` column identifies a unique sensor. Sensor readings should be recorded every 15 minutes from every sensor. Are we missing any readings, and if so which ones? We will focus on readings that are separated by at least 30 minutes or more; readings that are \[0-30) minutes apart are considered to be fine.

To answer this question you'll need to read up a bit on [SQL timestamps](https://www.postgresql.org/docs/current/datatype-datetime.html) and [Functions for manipulating datetime types](https://www.postgresql.org/docs/current/functions-datetime.html). Taking the time now to look at the documentation will save you time later on. Have a particular look at the following:
- The [date_trunc](https://www.postgresql.org/docs/current/functions-datetime.html#FUNCTIONS-DATETIME-TRUNC) function will quantize times to the nearest unit of your choosing. For example, to round the `time` field to the nearest minute you can say `date_trunc('minute', time)`. **You'll need to quantize to minutes right away before you worry about missing readings.**
- There are various ways to enter constant intervals of time as strings. For example, a 30 minute interval can be written as `interval '30 minutes'` or `'30 minutes'::interval`. See [date/time input](https://www.postgresql.org/docs/current/datatype-datetime.html#DATATYPE-INTERVAL-INPUT) for more info.
- You can do arithmetic on date/time types [as documented here](https://www.postgresql.org/docs/current/functions-datetime.html#FUNCTIONS-DATETIME-TRUNC). That will handle all the weird periodicities of clocks and calendars for you. Pay attention to the input and output types of these functions!
- You will need to use the [lag](https://www.geeksforgeeks.org/postgresql-lag-function/) function as the window function.


Create a view called `gaps` that augments the `data` schema with three columns:
- `lagtime` is the quantized time of the previous reading for that sensor (relative to the current row for a particular row)
- `lagvalue` is the value of the previous reading for that sensor
- `timediff` is the difference in quantized time between this reading and the previous reading

The view should only contain rows where `timediff` is **greater than or equal to 30 minutes**.

In [31]:
%%sql --save query_4a result_4a <<
CREATE OR REPLACE VIEW gaps AS
WITH unfiltered AS (SELECT *, 
    date_trunc('minute', LAG(time, 1) OVER (PARTITION BY id ORDER BY time)) AS lagtime,
    LAG(value, 1) OVER (PARTITION BY id ORDER BY time) AS lagvalue,
    date_trunc('minute', time) - date_trunc('minute', LAG(time, 1) OVER (PARTITION BY id ORDER BY time)) AS timediff
FROM data)
SELECT * FROM unfiltered WHERE timediff >= INTERVAL '30 minutes';
SELECT * FROM gaps ORDER BY id, time LIMIT 10;

Running query in 'postgresql://jovyan@127.0.0.1:5432/ucb_buildings'

10 rows affected.

In [32]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_4a = %sqlcmd snippets query_4a
grading_util.save_results("result_4a", query_4a, result_4a)
result_4a

time,id,value,lagtime,lagvalue,timediff
2018-06-13 09:15:00+00:00,a3d47b1a-985e-5395-a6ee-719dad9b580f,0.056,2018-06-13 08:45:00+00:00,0.056,0:30:00
2018-06-09 04:15:00+00:00,a3e8e405-0eed-59b6-8747-fe892a6f93de,280.8,2018-06-09 03:45:00+00:00,281.6,0:30:00
2018-06-13 09:15:00+00:00,a46fb790-028c-5d17-a3e6-7d08daec0c03,281.46,2018-06-13 08:45:00+00:00,281.39,0:30:00
2018-06-09 04:15:00+00:00,a470ac6d-d448-522b-9b74-8ed27f24de25,0.004,2018-06-09 03:45:00+00:00,0.006,0:30:00
2018-06-07 00:45:02+00:00,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,27.273,2018-06-07 00:15:00+00:00,30.0,0:30:00
2018-06-07 01:30:01+00:00,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,27.273,2018-06-07 01:00:00+00:00,23.077,0:30:00
2018-06-07 17:45:02+00:00,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,21.429,2018-06-07 17:15:00+00:00,25.0,0:30:00
2018-06-07 18:45:07+00:00,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,25.0,2018-06-07 18:15:00+00:00,30.0,0:30:00
2018-06-07 19:15:03+00:00,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,20.0,2018-06-07 18:45:00+00:00,25.0,0:30:00
2018-06-07 19:45:05+00:00,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,23.077,2018-06-07 19:15:00+00:00,20.0,0:30:00


In [33]:
grader.check("q4a")

q4a results: All test cases passed!

### Question 4b: Creating tuples for the missing readings
Now we need to manufacture new tuples to fill in the gaps. For example, if you had a tuple from id `abc` timestamped at 1PM today and the next tuple in time from `abc` was timestamped at 1:45PM, you'll need to manufacture two new tuples with id `abc` and `NULL` values: one timestamped at 1:15PM and another timestamped at 1:30PM. We will worry about replacing the `NULL` values in the next step.

To manufacture tuples not related to stored data in the database, we'll need to use a *table-valued function*. The table-valued function we want here is `generate_series` [(documented here)](https://www.postgresql.org/docs/current/functions-srf.html), which we will use to generate *and sequentially timestamp* the right number of tuples to match the number of tuples we found missing.

To get a feel for `generate_series`, consider the following simple query that generates a table of integers with intervals of size 3 between them.

In [34]:
%%sql
SELECT *
  FROM generate_series(1, 10, 3);

Running query in 'postgresql://jovyan@127.0.0.1:5432/ucb_buildings'

4 rows affected.

generate_series
1
4
7
10


Now, we can use `generate_series` in a `LATERAL JOIN` query: for each tuple on the left of the `LATERAL` it will produce a series based on the values of that tuple. So for example, we can generate 2 tuples for each tuple of `uc_locations` as follows:

In [35]:
%%sql
SELECT loc_id, loc_name, length(loc_name), newval
  FROM uc_locations u, 
       LATERAL generate_series(length(loc_name), length(loc_name) + 2, 2) AS newval;

Running query in 'postgresql://jovyan@127.0.0.1:5432/ucb_buildings'

24 rows affected.

loc_id,loc_name,length,newval
1,AG FIELD STAT,13,13
1,AG FIELD STAT,13,15
2,BERKELEY,8,8
2,BERKELEY,8,10
3,DAVIS,5,5
3,DAVIS,5,7
4,IRVINE,6,6
4,IRVINE,6,8
5,LOS ANGELES,11,11
5,LOS ANGELES,11,13


Notice how the 2 values it generates are the length of the `loc_name`, and the length + 2. You might want to play with the query above to make sure you understand the documentation for `generate_series` and `LATERAL`.

Ok, on to your task!

Create a view `complete` that contains the tuples from `data` as well as new tuples that fill in any gaps greater than or equal to 30 minutes. Each gap should be filled by adding tuples in increments of 15 minutes from the *start* of the gap, **with `NULL` as the value**. You probably want to use your `gaps` view as well as `generate_series` to do this!

**Hints:** 
- The lower and upper bounds in generate_series (in pseudocode) should be `(lagtime + 15 minutes, time - 15 minutes)`.
- You may want to use UNION ALL to combine your data.

In [36]:
%%sql --save query_4b result_4b <<
CREATE OR REPLACE VIEW complete AS
WITH fill AS (SELECT id, new_time
              FROM gaps,
              LATERAL generate_series(lagtime + INTERVAL '15 minutes', time - INTERVAL '15 minutes', INTERVAL '15 minutes') AS new_time)
SELECT  time, id, value FROM data UNION ALL SELECT new_time AS time, id, NULL AS value FROM fill;
SELECT * FROM complete ORDER BY id, time LIMIT 100;

Running query in 'postgresql://jovyan@127.0.0.1:5432/ucb_buildings'

100 rows affected.

In [37]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_4b = %sqlcmd snippets query_4b
grading_util.save_results("result_4b", query_4b, result_4b)
result_4b

time,id,value
2018-06-07 00:00:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65085.99
2018-06-07 00:15:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65086.16
2018-06-07 00:30:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65086.35
2018-06-07 00:45:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65086.52
2018-06-07 01:00:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65086.71
2018-06-07 01:15:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65086.89
2018-06-07 01:30:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65087.07
2018-06-07 01:45:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65087.25
2018-06-07 02:00:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65087.43
2018-06-07 02:15:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65087.62


In [38]:
grader.check("q4b")

q4b results: All test cases passed!

### Question 4c: Linear Interpolation
*Note: If you struggled with the previous subparts of this problem, you can use our table `complete_provided` instead of your `complete` table in this subpart.*

Now, given the `complete` view or the `complete_provided` table, the remaining task is to do linear interpolation to fill in the missing values we manufactured in Step 2. We have code from Lecture 16 (Data Cleaning II) we can adapt from and use here! In particular, your database already includes the UDA (User-Defined Aggregate) `coalesce_agg` we used in lecture (you can use it directly, there's no need to redefine it).

But note that in Lecture 15's example of linear interpolation we had a field that was used to order *all* the records in the table. In contrast, here the ordering we care about is the series of `time` for each sensor `id` *independently*. **You will need to make minor changes to adapt the linear interpolation code from class to work here.**

Create a view `likely_data` that contains all the tuples from `complete`, with an additional column called `interpolated` that contains a copy of `value` if it is non-NULL, otherwise an interpolated value based on linear interpolation **per sensor id over time**. Please **retain all additional columns created from forward and backward passes**. (There should be 10 columns in the view—the order of the columns does not matter.) The three cells below correspond to the forward, backward, and final passes from lecture.

In [39]:
%%sql
CREATE OR REPLACE VIEW forward AS
    SELECT time, id, value, 
        SUM(CASE WHEN value IS NULL THEN 0 ELSE 1 END) OVER (PARTITION BY id ORDER BY time) AS run,
        coalesce_agg(value) OVER (PARTITION BY id ORDER BY time) AS run_start,
        CASE
            WHEN value IS NULL THEN LEAD(value, 1) OVER (PARTITION BY id ORDER BY time)
            ELSE NULL
            END AS next_val
    FROM complete_provided;

Running query in 'postgresql://jovyan@127.0.0.1:5432/ucb_buildings'

++
||
++
++

In [40]:
%%sql
CREATE OR REPLACE VIEW backward AS
    SELECT *, CASE
                  WHEN value IS NOT NULL THEN value
                  ELSE coalesce_agg(next_val) OVER (PARTITION BY id, run ORDER BY time DESC)
                  END AS run_end,
              COUNT(*) OVER (PARTITION BY id, run) AS run_size,
              -1 + (RANK() OVER (PARTITION BY id, run ORDER BY time)) AS run_rank
    FROM forward;

Running query in 'postgresql://jovyan@127.0.0.1:5432/ucb_buildings'

++
||
++
++

In [41]:
%%sql --save query_4c result_4c <<
CREATE OR REPLACE VIEW likely_data AS
SELECT *, run_start + run_rank*((run_end-run_start)/(run_size)) AS interpolated
    FROM backward

;
SELECT * FROM likely_data WHERE run_size > 2 ORDER BY id, time LIMIT 100;

Running query in 'postgresql://jovyan@127.0.0.1:5432/ucb_buildings'

100 rows affected.

In [42]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_4c = %sqlcmd snippets query_4c
grading_util.save_results("result_4c", query_4c, result_4c)
result_4c

time,id,value,run,run_start,next_val,run_end,run_size,run_rank,interpolated
2018-06-08 10:00:02+00:00,a49cd747-43c3-536d-b3ee-ec9c087b9317,1.7230274441963545,137,1.7230274441963545,None,1.7230274441963545,3,0,1.7230274441963545
2018-06-08 10:15:00+00:00,a49cd747-43c3-536d-b3ee-ec9c087b9317,None,137,1.7230274441963545,None,5.372979444921953,3,1,2.939678111104887
2018-06-08 10:30:00+00:00,a49cd747-43c3-536d-b3ee-ec9c087b9317,None,137,1.7230274441963545,5.372979444921953,5.372979444921953,3,2,4.156328778013419
2018-06-08 19:45:03+00:00,a49cd747-43c3-536d-b3ee-ec9c087b9317,0.6144687255293491,174,0.6144687255293491,None,0.6144687255293491,3,0,0.6144687255293491
2018-06-08 20:00:00+00:00,a49cd747-43c3-536d-b3ee-ec9c087b9317,None,174,0.6144687255293491,None,8.10343967255239,3,1,3.1107923745370294
2018-06-08 20:15:00+00:00,a49cd747-43c3-536d-b3ee-ec9c087b9317,None,174,0.6144687255293491,8.10343967255239,8.10343967255239,3,2,5.60711602354471
2018-06-09 01:00:03+00:00,a49cd747-43c3-536d-b3ee-ec9c087b9317,2.196116776472188,193,2.196116776472188,None,2.196116776472188,3,0,2.196116776472188
2018-06-09 01:15:00+00:00,a49cd747-43c3-536d-b3ee-ec9c087b9317,None,193,2.196116776472188,None,1.4019035586474615,3,1,1.931379037197279
2018-06-09 01:30:00+00:00,a49cd747-43c3-536d-b3ee-ec9c087b9317,None,193,2.196116776472188,1.4019035586474615,1.4019035586474615,3,2,1.6666412979223701
2018-06-10 05:00:02+00:00,a49cd747-43c3-536d-b3ee-ec9c087b9317,4.692789500399002,304,4.692789500399002,None,4.692789500399002,3,0,4.692789500399002


In [43]:
grader.check("q4c")

q4c results: All test cases passed!

## Question 5: Granularity Transforms
In this question we will write a roll-up query on an ontology. This requires a bit of background explanation.

## [Tutorial, read] 

### The Brick Ontology

An ontology is a way of showing the properties of a subject area and how they are related, by defining a set of terms and relational expressions that represent the entities in that subject area. 

The [Software-Defined Buildings](http://sdb.cs.berkeley.edu/sdb/) research project at Berkeley has led the development of a standard ontology for building metadata called [Brick](https://docs.brickschema.org/intro.html) that is getting a fair bit of attention in the world of IoT. Like many ontologies, it is represented as triples `(subject, predicate, object)`. In our database, the Brick ontology has been stored in a table called `ontology`.

### The `SubClassOf` Predicate and the `Sensor` Class

We are interested in readings from different classes of sensor devices. More specifically, we are interested in rows from the `metadata` table whose `metadata.class` entry maps to an `ontology` subject $s$, and that subject is in an ontology tuple `(`$s$`, http://www.w3.org/2000/01/rdf-schema#subClassOf, https://brickschema.org/schema/Brick#Sensor)`. Then we know that the sensor in `metadata` belongs to a sub-class of `Sensor`. A Subject is connected via Predicate to an Object. For example, a sensor child is connected via is `subClassOf` to its sensor parent. Unlike other trees that typically go from parent node down to leaf node, we are traversing from child node **up** to parent node.

The diagram below shows a few of the `subject`s and `object`s from `ontology` in ovals. There is a dark arrow between two ovals if there is a corresponding row in `ontology`. The immediate sub-classes of `Sensor` in the diagram are shown in yellow; we'll call them "Sensor children".

Thus, a sensor child is a sensor that is in the subclass of ***Sensor***. So, ***Sensor*** (Upper-cased) is it's parent. All of the children underneath ***Sensor*** and ***Sensor's*** children are also sensor children (like a family tree, where grandparent is ***Sensor***, parent is one of ***Sensor's*** children, and grandchild is also one of ***Sensor's*** children, and so is great-grandchild and so on). 
| Parent

<img src="data/subClass.png">

This image is available as `data/subClass.png`, and can be made available full-screen for ease of access.

### The `transitive_subClassOf` Relation

The `transitive_subClassOf` materialized view is the set of edges that compose the ontology graph, i.e., all dark black and light green arrows on the diagram. It contains tuples of the form `(object, subject, hops, path)` where subject and object are connected transitively in `ontology` via one or more subClassOf predicates described above. `path` is a Postgres array type that shows the transitive path of class names through the ontology from subject to object, and `hops` is the length of that path. Run the next cell to see the first 4 rows of `transitive_subClassOf`:

In [44]:
%sql SELECT * FROM transitive_subClassOf LIMIT 4;

Running query in 'postgresql://jovyan@127.0.0.1:5432/ucb_buildings'

4 rows affected.

object,subject,hops,path
https://brickschema.org/schema/Brick#Room,https://brickschema.org/schema/Brick#Ablutions_Room,1,"['https://brickschema.org/schema/Brick#Ablutions_Room', 'https://brickschema.org/schema/Brick#Room']"
ub1bL18C21,https://brickschema.org/schema/Brick#Ablutions_Room,1,"['https://brickschema.org/schema/Brick#Ablutions_Room', 'ub1bL18C21']"
https://brickschema.org/schema/Brick#Chiller,https://brickschema.org/schema/Brick#Absorption_Chiller,1,"['https://brickschema.org/schema/Brick#Absorption_Chiller', 'https://brickschema.org/schema/Brick#Chiller']"
ub1bL51C21,https://brickschema.org/schema/Brick#Absorption_Chiller,1,"['https://brickschema.org/schema/Brick#Absorption_Chiller', 'ub1bL51C21']"


The `transitive_subClassOf` edge graph is also called a [transitive closure](https://en.wikipedia.org/wiki/Transitive_closure) of the `subClassOf` predicate, because it represents that subclasses of subclasses of classes are sensors, and so on.

We have already materialized this view for you, but to clarify its structure, we formed transitive “chains” by joining ontology with itself. For example, we could have made the materialized view `transitive_2edge_subClassOf` with the below command (Note that this materialized view does **not** exist.):
```sql
CREATE MATERIALIZED VIEW transitive_2edge_subClassOf AS
    SELECT o1.subject, o2.object 
      FROM ontology o1 INNER JOIN ontology o2 ON o1.object = o2.subject
     WHERE o1.predicate = 'http://www.w3.org/2000/01/rdf-schema#subClassOf'
       AND o2.predicate = 'http://www.w3.org/2000/01/rdf-schema#subClassOf';
``` 

Extending this example, in order to compute edges of length 3 we would have needed to join 3 references to `ontology`, and so on. To form all chains of arbitrary length requires the use of a recursive query—something we haven't learned in this class. Again, we have already done this for you in the materialized view `transitive_subClassOf` that provides the result of that recursive query. 

*Just for fun: If you're curious about the recursive query that computes this view, you can issue the command `\d+ transitive_subClassOf` to Postgres. You may also want to read the documentation for SQL's [WITH RECURSIVE](https://www.postgresql.org/docs/9.1/queries-with.html) clause as implemented in Postgres.*

### Question 5a

We want to check the graph properties of the `subClassOf` predicate. It would be confusing if the `subClassOf` predicate had cycles!nt

Write a query on `transitive_subClassOf` to check for cycles. Ask yourself this: what property in `transitive_subClassOf` would be indicative of a cycle? Your query should return one row of one boolean column: `true` if the predicate has cycles, `false` otherwise.

**Hint:** You may find `EXISTS` to be useful here.

In [45]:
%%sql --save query_5a result_5a <<
SELECT CASE
    WHEN EXISTS (
        SELECT 1
        FROM transitive_subClassOf
        WHERE object = subject
    ) THEN TRUE
    ELSE FALSE
END AS has_cycles;

Running query in 'postgresql://jovyan@127.0.0.1:5432/ucb_buildings'

1 rows affected.

In [46]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_5a = %sqlcmd snippets query_5a
grading_util.save_results("result_5a", query_5a, result_5a)
result_5a

has_cycles
False


In [47]:
grader.check("q5a")

q5a results: All test cases passed!

### Question 5b

Assuming it's not cyclic, the next question is whether the `subClassOf` predicate forms *tree-shaped* connections only. The signature of a tree is that each node has at most one outbound edge (pointing to its "parent"). If any node has multiple outbound edges, the predicate forms a more general directed acyclic graph (a DAG). So we are looking to see if each subject is in a `subClassOf` predicate *with at most one object* (the single parent in the tree). 

Write a query that returns `true` if *each* subject is in a `subClassOf` predicate with at most one object, and `false` otherwise.

**Hint:** Refer to how we created `transitive_2edge_subClassOf` in the tutorial to see how we check if something is in a `subClassOf` predicate.

In [48]:
%%sql --save query_5b result_5b <<
WITH object_count AS (
SELECT subject, COUNT(object) AS count FROM transitive_subClassOf WHERE hops = 1 GROUP BY subject)
SELECT CASE
        WHEN SUM(count) != COUNT(count) THEN FALSE
        ELSE TRUE
        END AS check
        FROM object_count;

Running query in 'postgresql://jovyan@127.0.0.1:5432/ucb_buildings'

1 rows affected.

In [49]:
%%sql
WITH object_ont AS (
SELECT subject, COUNT(object) AS count FROM ontology WHERE predicate = 'http://www.w3.org/2000/01/rdf-schema#subClassOf' GROUP BY subject)
SELECT CASE
        WHEN SUM(count) != COUNT(count) THEN FALSE
        ELSE TRUE
        END AS check
        FROM object_ont;

Running query in 'postgresql://jovyan@127.0.0.1:5432/ucb_buildings'

1 rows affected.

check
False


In [50]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_5b = %sqlcmd snippets query_5b
grading_util.save_results("result_5b", query_5b, result_5b)
result_5b

check
False


In [51]:
grader.check("q5b")

q5b results: All test cases passed!

### Question 5c

Now that we understand the graph properties of the ontology, let's use the ontology to do a roll-up as intended. Remember from [lecture 13](https://docs.google.com/presentation/d/19Nsksm89k8NBap9rwINJJFJeZCDSmF-feM6o0mz868c/edit?usp=sharing) that a roll-up is transforming to coarser grain (e.g., go up in a hierarchy). We're interested in the number of unique sensor `id`s from `metadata` that are transitively in subclasses of each "Sensor child" class. To compute this, you will have to associate each `metadata.id` with a matching `brickclass` *(if there is one!)*, and use the `transitive_subClassOf` view to identify all "Sensor children" for which the matching `brickclass` is transitively in a `subClass`. (If the `subClass` predicate is a DAG, then each `id` should be counted for *all* the direct subclasses of `Sensor` that it's transitively underneath.)

Write a query that returns tuples of the form `(sensor_child, count)` that returns for each "Sensor child" the count of **distinct** `metadata.id` entries that are subclasses of that "Sensor child" class. Only output tuples that have a matching `brickclass`.

**Hints:**
- We are interested in "the count of distinct metadata.id entries that are subclasses of that "Sensor child" class". That means we are interested in all the sensors that are children of the sensors that have "Sensor" as its parent (these are the nodes in yellow in our diagram). It may be helpful to first create a CTE where you identify this set of unique sensor children according to these two properties (parent and predicate to specify that we are interested in the specific parent-child relationship and not other types of relationships) using information obtainable from the ontology table.

Refer to `data/schema.png` for a refresher on the schema. This will be helpful to identify:
- how you can associate sensors in `metadata` with a matching `brickclass` via `mapping`
- how you can use the `transitive_subClassOf` view which has attributes `object` and `subject` just like `ontology` to match "Sensor children" with `brickclass`.


In [52]:
%%sql --save query_5c result_5c <<
WITH sensor_children AS (
    SELECT subject AS sensor_childs FROM ontology WHERE object = 'https://brickschema.org/schema/Brick#Sensor' 
    AND predicate = 'http://www.w3.org/2000/01/rdf-schema#subClassOf'),
sensor_grandchildren AS (
    SELECT subject AS sensor_gchilds, object AS sensor_kid FROM transitive_subClassOf WHERE object IN (SELECT * FROM sensor_children))
SELECT s.sensor_kid, COUNT(DISTINCT(m.id)) FROM metadata m JOIN mapping map ON m.class = map.rawname JOIN sensor_grandchildren s ON map.brickclass = s.sensor_gchilds GROUP BY s.sensor_kid; 
    

Running query in 'postgresql://jovyan@127.0.0.1:5432/ucb_buildings'

2 rows affected.

In [53]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_5c = %sqlcmd snippets query_5c
grading_util.save_results("result_5c", query_5c, result_5c)
result_5c

sensor_kid,count
https://brickschema.org/schema/Brick#Demand_Sensor,92
https://brickschema.org/schema/Brick#Power_Sensor,3106


In [54]:
grader.check("q5c")

q5c results: All test cases passed!

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Congratulations! You have finished Project 3.

Run the following cell to zip and download the results of your queries. You will also need to run the export cell at the end of the notebook.

**Please save your notebook before exporting (this is a good time to do it!)** Otherwise, we may not be able to export your written responses to `proj3.pdf`. We will not be accepting regrade requests for failure to render written responses.

**For your submission on Gradescope, you will only need to submit the single `proj3.zip` file generated by the export cell.** Please ensure that your submission `proj3.zip` file includes `proj3.pdf`, `proj3.ipynb`, and `results.zip`. 

**Please ensure that public tests pass upon submission.** It is your responsibility to wait until the autograder finishes running. We will not be accepting regrade requests for submission issues.

**Common submission issues:** You MUST submit the generated zip file to the autograder. However, Safari is known to automatically unzip files upon downloading. You can fix this by going into Safari preferences, and deselect the box with the text "Open safe files after downloading" under the "General" tab. If you experience issues with downloading via clicking on the link, you can also navigate to the project 3 directory within JupyterHub (remove `proj3.ipynb` from the url), and manually download the generated zip files. Please post on Ed if you encounter any other submission issues.

Run the following cell to zip and download the results of your queries. You will also need to run the export cell at the end of the notebook.

In [55]:
grading_util.prepare_submission_and_cleanup()  # builds results.zip

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [56]:
grader.check_all()

q1b results: All test cases passed!

q1c results: All test cases passed!

q2a results: All test cases passed!

q2b results: All test cases passed!

q3b results: All test cases passed!

q3c results: All test cases passed!

q3e results: All test cases passed!

q4a results: All test cases passed!

q4b results: All test cases passed!

q4c results: All test cases passed!

q5a results: All test cases passed!

q5b results: All test cases passed!

q5c results: All test cases passed!

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(files=['results.zip'])